In [11]:
import os, json
from azureml.core import Workspace, Experiment, Dataset, Datastore, ScriptRunConfig, Environment
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DockerConfiguration

workspace = Workspace.from_config()

In [12]:
#datastore = Datastore(workspace, "demostore")
datastore = workspace.get_default_datastore()
input_data = Dataset.File.from_files(datastore.path('mlops_timeseries/mlops_forecast_data2013.csv')).as_named_input('input_ds').as_mount()


In [13]:
myenv = Environment.from_conda_specification(name="arima-env", file_path="arima-env.yml")
docker_config = DockerConfiguration(use_docker=True)

aml_compute = 'demo-cpucluster1'  # <== Compute Cluster Name
source_directory = './scripts'
script='train.py'

In [14]:
model_name = 'arima-model-v3'
experiment_name = 'arima-mlops-remote-v3'

In [15]:
src = ScriptRunConfig(source_directory=source_directory,
                      script=script,
                      arguments =[
                                  '--model_name', model_name,
                                  '--input', input_data, 
                                 ],
                      compute_target=aml_compute,
                      environment=myenv,
                      docker_runtime_config=docker_config)

In [16]:
experiment = Experiment(workspace = workspace, name = experiment_name)

run = experiment.submit(config=src)
run

Experiment,Id,Type,Status,Details Page,Docs Page
arima-mlops-remote-v3,arima-mlops-remote-v3_1650758172_65c70524,azureml.scriptrun,Queued,Link to Azure Machine Learning studio,Link to Documentation


In [17]:
run.wait_for_completion(show_output=True, wait_post_processing=True)

RunId: arima-mlops-remote-v3_1650758172_65c70524
Web View: https://ml.azure.com/runs/arima-mlops-remote-v3_1650758172_65c70524?wsid=/subscriptions/8ee4fda7-36cd-41c4-baee-f7d48bb5f2b8/resourcegroups/rg_amlworkspace/workspaces/demo-mlworkspace01&tid=72f988bf-86f1-41af-91ab-2d7cd011db47

Execution Summary
RunId: arima-mlops-remote-v3_1650758172_65c70524
Web View: https://ml.azure.com/runs/arima-mlops-remote-v3_1650758172_65c70524?wsid=/subscriptions/8ee4fda7-36cd-41c4-baee-f7d48bb5f2b8/resourcegroups/rg_amlworkspace/workspaces/demo-mlworkspace01&tid=72f988bf-86f1-41af-91ab-2d7cd011db47



{'runId': 'arima-mlops-remote-v3_1650758172_65c70524',
 'target': 'demo-cpucluster1',
 'status': 'Completed',
 'startTimeUtc': '2022-04-23T23:56:31.04462Z',
 'endTimeUtc': '2022-04-23T23:56:50.319159Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  'ContentSnapshotId': '0a855dfb-bdcf-40b7-989a-37dcae95a202',
  'azureml.git.repository_uri': 'https://github.com/notanaha/oh4ml-lite.git',
  'mlflow.source.git.repoURL': 'https://github.com/notanaha/oh4ml-lite.git',
  'azureml.git.branch': 'main',
  'mlflow.source.git.branch': 'main',
  'azureml.git.commit': '933567d2de7a80f35dd29f9b3237dd35057b80f0',
  'mlflow.source.git.commit': '933567d2de7a80f35dd29f9b3237dd35057b80f0',
  'azureml.git.dirty': 'True',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': '5e2262c6-57bf-4ef5-896e-8c7b850ac5a0'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'input_ds'

In [18]:
os.makedirs('./models', exist_ok=True)
run.download_file('outputs/models/' + 'arima_model.pkl', output_file_path='models')
run.download_file('outputs/metric.json', output_file_path='scripts')

In [19]:
run.get_file_names()

['outputs/metric.json',
 'outputs/models/arima_model.pkl',
 'outputs/pred.png',
 'outputs/res.png',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/data_capability/data-capability.log',
 'system_logs/data_capability/rslex.log',
 'system_logs/data_capability/rslex.log.2022-04-23-23',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/lifecycler/vm-bootstrapper.log',
 'user_logs/std_log.txt']

In [20]:
#run.register_model('arima-model-v3', model_path='outputs/arima_model.pkl', tags=run.get_metrics(), description='arima-model-v2')